## Neural Network Heart Disease Detection using FHE

#### This demo uses the Heart Disease dataset, originally taken from: https://archive.ics.uci.edu/ml/datasets/Heart+Disease

This database contains 76 attributes, but all published experiments refer to using a subset of 14 of them. See citation at the bottom of the page.

In [ ]:
# Imports and misc. init
import numpy as np
import h5py
import utils
import pyhelayers

utils.verify_memory()

input_dir = utils.get_data_sets_dir() + "/net_medical/" # for the model and data

# Load test data and labels
with h5py.File(input_dir + "x_test.h5") as f:
    x_test = np.array(f["x_test"])
with h5py.File(input_dir + "y_test.h5") as f:
    y_test = np.array(f["y_test"])
plain_samples, labels = utils.extract_batch(x_test, y_test, 16, 0)

# Load plain model
hyper_params = pyhelayers.PlainModelHyperParams()
nnp = pyhelayers.NeuralNetPlain()
nnp.init_from_files(hyper_params, [input_dir + "model.json", input_dir + "model.h5"])
print("loaded plain model")

# Automatic optimziations
he_run_req = pyhelayers.HeRunRequirements()
he_run_req.set_he_context_options([pyhelayers.DefaultContext()])
he_run_req.optimize_for_batch_size(16)
profile = pyhelayers.HeModel.compile(nnp, he_run_req)
batch_size = profile.get_optimal_batch_size()
context = pyhelayers.HeModel.create_context(profile)
print('HE Context ready. Batch size=',batch_size)

# Create encrypted network
nn = pyhelayers.NeuralNet(context)
nn.encode_encrypt(nnp, profile)

# Encrypt input samples
iop = nn.create_io_processor()
samples = pyhelayers.EncryptedData(context)
iop.encode_encrypt_inputs_for_predict(samples, [plain_samples])

# MAIN PART:
# Perform inference under encryption 
utils.start_timer()
predictions = pyhelayers.EncryptedData(context)
nn.predict(predictions, samples)
duration=utils.end_timer('predict')
utils.report_duration('predict per sample',duration/batch_size)

# Decrypt results
plain_predictions = iop.decrypt_decode_output(predictions)

print('\nclassification results')
print('=========================================')
for label,pred in zip(labels,plain_predictions):
    print('Label:',('Healthy' if label==0 else 'Should talk with a Dr.'),end=', ')
    print('Prediction:',('Healthy' if pred[0]<0.5 else 'Should talk with a Dr.'))    

<br>

Citation:

<sub><sup> 1.	https://archive.ics.uci.edu/ml/datasets/Heart+Disease </sup></sub>

<sub><sup> 2.	Detrano, R., Janosi, A., Steinbrunn, W., Pfisterer, M., Schmid, J., Sandhu, S., Guppy, K., Lee, S., & Froelicher, V. (1989). International application of a new probability algorithm for the diagnosis of coronary artery disease. American Journal of Cardiology, 64,304--310.</sup></sub>

<sub><sup> 3.	David W. Aha & Dennis Kibler. "Instance-based prediction of heart-disease presence with the Cleveland database."</sup></sub>

<sub><sup> 4.	Gennari, J.H., Langley, P, & Fisher, D. (1989). Models of incremental concept formation. Artificial Intelligence, 40, 11--61.
 </sup></sub>